In [6]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

### Choose a time partition

In [2]:
tp = 1

Load Database

In [3]:
data  = pd.read_csv(f'datos\counted\d{tp}_counted.csv', index_col=0)

In [4]:
data.columns

Index(['avisoid', 'empresaid', 'avisofechapublicacion', 'avisovacante',
       'mostrarsueldo', 'avisoexperiencia', 'expiracion', 'dias',
       'avisorepublicacion', 'avisocargo', 'areanombre',
       'actividadempresanombre', 'avisocuerpo', 'disponibilidadnombre',
       'avisoduracioncont', 'avisolugartrabajo', 'gradoescolarnombre',
       'situacionestudios', 'avisorequisitos', 'carreras', 'estado',
       'endpagado_o_gratuito', 'nivelnombre', 'carreras_array',
       'sueldoestimado', '_merge', 'avcu_b1', 'avca_b1', 'avre_b1', 'avcu_b2',
       'avca_b2', 'avre_b2', 'avcu_b3', 'avca_b3', 'avre_b3', 'avcu_b4',
       'avca_b4', 'avre_b4', 'teletrabajo_', 'bucket1', 'bucket2', 'bucket3',
       'bucket4', 'wfh'],
      dtype='object')

In [63]:
data['avisolugartrabajo']

0                                           NaN
1                                           NaN
2                                           NaN
3                                           NaN
4                                           NaN
                           ...                 
2301634                                Valdivia
2301635                                     NaN
2301636                              Concepción
2301637     Santiago, Metropolitana de Santiago
2301638    ohiggins 133 santa leonor talcahuano
Name: avisolugartrabajo, Length: 2301639, dtype: object

In [17]:
data['avisofechapublicacion'] = pd.to_datetime(data['avisofechapublicacion'])

### Choose a window size in days

In [57]:
days_before = 30
days_after = 30

In [84]:
dates = data['avisofechapublicacion'].unique()

In [90]:
dates = pd.Series(data['avisofechapublicacion'].unique())

In [98]:
data['duplicates'] = None

In [91]:
window_before = timedelta(days=days_before)
window_after = timedelta(days=days_after)

start_date = dates[0] - window_before
final_date = dates[0] + window_after

newData = data[(data['avisofechapublicacion'] >= start_date) & (data['avisofechapublicacion'] < final_date)]
duplicates = newData[newData.duplicated(subset=['empresaid', 'avisocuerpo', 'avisolugartrabajo'], keep=False)]

data['duplicates'] = data.apply(lambda row: row in newData.to_records(index=False), axis=1)


In [102]:
duplicates.index

Int64Index([422784, 449436, 519981, 553176, 627666, 627932, 627933, 630278,
            631123, 631146,
            ...
            959800, 959802, 959803, 959807, 959808, 959830, 959833, 959836,
            959844, 959850],
           dtype='int64', length=8316)

In [106]:
data['duplicates'] = data.index.isin(duplicates.index).sum()

In [72]:
data.loc

422784    TNT Express, prestigiosa empresa de transporte...
449436    Importante Institución de Educación Superior, ...
519981    DTS CALL CENTER REQUIERE EJECUTIVOS DE ATENCIO...
553176    Importante empresa de Outsoursing de servicios...
627666    Importante empresa de call center, busca Ejecu...
                                ...                        
959830    Empresa de transportes busca incorporar a prac...
959833    Rotter & Krauss, requiere alumno en práctica d...
959836    Empresa Rotter & Krauss requiere administrativ...
959844    ACHS ofrece un cupo para alumno de Prevención ...
959850    ACHS ofrece un cupo para alumno de Prevención ...
Name: avisocuerpo, Length: 8316, dtype: object

In [73]:
duplicates['avisocuerpo'][959850]

'ACHS ofrece un cupo para alumno de Prevención de Riesgos para formar parte de su equipo.\r\nLos interesados deben tener disponibilidad de aprender y motivación por aplicar sus conocimientos.'

In [74]:
duplicates['avisocuerpo'][959844]

'ACHS ofrece un cupo para alumno de Prevención de Riesgos para formar parte de su equipo.\r\nLos interesados deben tener disponibilidad de aprender y motivación por aplicar sus conocimientos.'

In [75]:
duplicates['empresaid'][959844]

7383

In [76]:
duplicates['empresaid'][959850]

7383

In [79]:
duplicates['avisolugartrabajo'][959844]

'Temuco'

In [78]:
duplicates['avisolugartrabajo'][959850]

'Temuco'